# Import Packages

In [1]:
import tweepy
from textblob import TextBlob
import emoji
import re
import string
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Data Collection

In [2]:
ACCESS_TOKEN = "2378032825-QNQ4zETCM2hcIuP80deSHGdTXcrzE7vysvkdcNz"
ACCESS_TOKEN_SECRET = "JIoaBDifICUapIXWGE1x636itkIrHHxs7he0WwiClY6kv"
CONSUMER_KEY = "xU5CZUCtq2jNiopmogUvegudT"
CONSUMER_SECRET = "goL8kGtG5MMe5oKjivh3MT6Ic88iajrjqJ26NpJj5upiJFB3Pa"

In [3]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [4]:
api = tweepy.API(auth)

In [5]:
query = 'wordscapes' # set 'wordscapes' as a search query
max_tweets = 1000

In [6]:
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query, tweet_mode="extended").items(max_tweets)]

Helpful resource: [structure-of-tweepy-status-object.json](https://gist.github.com/dev-techmoe/ef676cdd03ac47ac503e856282077bf2)

# Data Parsing

In [7]:
def get_tweet_type(tweet):
    """ Get tweet type: tweet, retweet, quote, reply
    Details for tweet types:  https://gwu-libraries.github.io/sfm-ui/posts/2016-11-10-twitter-interaction
    """
    if hasattr(tweet, 'retweeted_status'):
        return "retweet"
    elif hasattr(tweet, 'quoted_status'):
        return "quote"
    elif tweet.in_reply_to_status_id_str is not None:
        return "reply"
    else:
        return "tweet"

In [8]:
def replace_urls(in_string, replacement=None):
    """Replace URLs in strings.

    Args:
        in_string (str): string to filter
        replacement (str or None): replacment text. defaults to ''

    Returns:
        str
    """
    replacement = '' 
    pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
    return re.sub(pattern, replacement, in_string)

In [9]:
def remove_emoji(in_string):
    return emoji.get_emoji_regexp().sub(u' ', in_string)

In [10]:
def get_original_tweet_id(tweet):
    """ Return original tweet id if tweet type is retweet. Otherwise, return its own id.
    """
    if get_tweet_type(tweet) == "retweet":
        return tweet.retweeted_status.id
    else:
        return tweet.id

In [11]:
data_df = pd.DataFrame([{"date": x.created_at,
                         "text": x.full_text,
                         "clean_text": remove_emoji(replace_urls(x.full_text)),
                         "tweet_id": x.id,
                         "if_retweet": get_original_tweet_id(x),
                         "user": x.user.screen_name,
                         "user_bio": x.user.description,
                         "user_id": x.user.id,
                         "at_mentions": [u["screen_name"] for u in x.entities['user_mentions']],
                         "hashtags": [u["text"] for u in x.entities['hashtags']],
                         "language": x.lang,
                         "type": get_tweet_type(x),
                         "retweet_count": x.retweet_count,
                         "favorite_count": x.favorite_count,
                         "polarity": TextBlob(x.full_text).sentiment.polarity} for x in searched_tweets]).set_index("date")

In [12]:
print(data_df.iloc[1])

at_mentions                                                      []
clean_text        i’m on level 350 on wordscapes in case y’all w...
favorite_count                                                    2
hashtags                                                         []
if_retweet                                      1177060826794139648
language                                                         en
polarity                                                          0
retweet_count                                                     0
text              i’m on level 350 on wordscapes in case y’all w...
tweet_id                                        1177060826794139648
type                                                          tweet
user                                                         CA22ON
user_bio          Graduate Assistant Coach for @ECUTigersMBB For...
user_id                                                   166237422
Name: 2019-09-26 03:22:24, dtype: object


In [13]:
data_df[["language","tweet_id"]].groupby("language").count()

,tweet_id
language,
en,231
fr,1
ht,1
in,1
it,1
pt,1
tl,36
und,7


In [14]:
data_df[["type","tweet_id"]].groupby("type").count()

,tweet_id
type,
quote,12
reply,52
retweet,18
tweet,197


In [15]:
def get_emotion(df):
    """
    polarity = 0: neutral;
    polarity > 0: positive;
    polarity < 0: negative.
    """
    if df['polarity'] == 0:
        return 'neutral'
    elif df['polarity'] > 0.0:
        return 'positive'
    else:
        return 'negative'

In [16]:
data_df['emotion'] = data_df.apply(get_emotion, axis=1)

In [17]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 279 entries, 2019-09-26 04:18:18 to 2019-09-16 17:36:48
Data columns (total 15 columns):
at_mentions       279 non-null object
clean_text        279 non-null object
favorite_count    279 non-null int64
hashtags          279 non-null object
if_retweet        279 non-null int64
language          279 non-null object
polarity          279 non-null float64
retweet_count     279 non-null int64
text              279 non-null object
tweet_id          279 non-null int64
type              279 non-null object
user              279 non-null object
user_bio          279 non-null object
user_id           279 non-null int64
emotion           279 non-null object
dtypes: float64(1), int64(5), object(9)
memory usage: 34.9+ KB


In [18]:
data_df.head()

,at_mentions,clean_text,favorite_count,hashtags,if_retweet,language,polarity,retweet_count,text,tweet_id,type,user,user_bio,user_id,emotion
date,,,,,,,,,,,,,,,
2019-09-26 04:18:18,[],I'm getting sick of the Wordscapes daily puzzl...,0,[],1177074894867259392,en,-0.357143,0,I'm getting sick of the Wordscapes daily puzzl...,1177074894867259392,tweet,A_Baca55,"I Have A Podcast, It's Ok",388541172,negative
2019-09-26 03:22:24,[],i’m on level 350 on wordscapes in case y’all w...,2,[],1177060826794139648,en,0.000000,0,i’m on level 350 on wordscapes in case y’all w...,1177060826794139648,tweet,CA22ON,Graduate Assistant Coach for @ECUTigersMBB For...,166237422,neutral
2019-09-26 02:51:54,[],I'm gonna finish my wordscapes daily puzzle an...,0,[],1177053151406542848,en,0.000000,0,I'm gonna finish my wordscapes daily puzzle an...,1177053151406542848,tweet,drcosimacormier,this Queen loves her beautiful Goddess 💖 #1 @...,4319425588,neutral
2019-09-26 02:37:41,[],"Ciara has played 2,000 games of Wordscapes and...",0,[],1177049574143352832,en,0.000000,0,"Ciara has played 2,000 games of Wordscapes and...",1177049574143352832,tweet,codyst3wart,,42117032,neutral
2019-09-26 00:40:01,[],Come play Wordscapes! It's a fun combination o...,0,[],1177019961233084416,en,0.375000,0,Come play Wordscapes! It's a fun combination o...,1177019961233084416,tweet,Naddez,Rockin' the Grandma Life #productreviewer #fam...,19444649,positive


In [19]:
retweet_list = []

In [20]:
for tweet in searched_tweets:
    if get_tweet_type(tweet) == "retweet":
        retweet_list.append(tweet)

In [21]:
retweet_df = pd.DataFrame([{"date": x.retweeted_status.created_at,
                         "text": x.retweeted_status.full_text,
                         "clean_text": remove_emoji(replace_urls(x.retweeted_status.full_text)),
                         "tweet_id": x.retweeted_status.id,
                         "user": x.retweeted_status.user.screen_name,
                         "user_bio": x.retweeted_status.user.description,
                         "user_id": x.retweeted_status.user.id,
                         "at_mentions": [u["screen_name"] for u in x.retweeted_status.entities['user_mentions']],
                         "hashtags": [u["text"] for u in x.retweeted_status.entities['hashtags']],
                         "language": x.retweeted_status.lang,
                         "type": get_tweet_type(x),
                         "retweet_count": x.retweeted_status.retweet_count,
                         "favorite_count": x.retweeted_status.favorite_count,
                         "polarity": TextBlob(x.retweeted_status.full_text).sentiment.polarity} for x in retweet_list]).set_index("date")

In [22]:
retweet_df['emotion'] = retweet_df.apply(get_emotion, axis=1)

In [23]:
retweet_df.head()

,at_mentions,clean_text,favorite_count,hashtags,language,polarity,retweet_count,text,tweet_id,type,user,user_bio,user_id,emotion
date,,,,,,,,,,,,,,
2019-09-25 12:23:06,[],me: *has 7 chapters of a book to read AND writ...,6,[],en,0.000000,1,me: *has 7 chapters of a book to read AND writ...,1176834509301518346,retweet,mmaddie_brookee,Thrower for Shawnee State. PTA ‘21. Exercise S...,1030099251991400448,neutral
2019-09-24 10:32:28,[],A palpable absence of excitement as the Circle...,26,[],en,0.017708,5,A palpable absence of excitement as the Circle...,1176444282070339584,retweet,BarbaraRich_law,"Barrister/mediator, law of mental incapacity, ...",586531888,positive
2019-09-24 10:32:28,[],A palpable absence of excitement as the Circle...,26,[],en,0.017708,5,A palpable absence of excitement as the Circle...,1176444282070339584,retweet,BarbaraRich_law,"Barrister/mediator, law of mental incapacity, ...",586531888,positive
2019-09-24 10:32:28,[],A palpable absence of excitement as the Circle...,26,[],en,0.017708,5,A palpable absence of excitement as the Circle...,1176444282070339584,retweet,BarbaraRich_law,"Barrister/mediator, law of mental incapacity, ...",586531888,positive
2019-09-24 10:32:28,[],A palpable absence of excitement as the Circle...,26,[],en,0.017708,5,A palpable absence of excitement as the Circle...,1176444282070339584,retweet,BarbaraRich_law,"Barrister/mediator, law of mental incapacity, ...",586531888,positive


In [24]:
retweet_df = retweet_df.drop_duplicates(subset='tweet_id', keep="last")

In [25]:
retweet_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11 entries, 2019-09-25 12:23:06 to 2019-09-16 23:10:01
Data columns (total 14 columns):
at_mentions       11 non-null object
clean_text        11 non-null object
favorite_count    11 non-null int64
hashtags          11 non-null object
language          11 non-null object
polarity          11 non-null float64
retweet_count     11 non-null int64
text              11 non-null object
tweet_id          11 non-null int64
type              11 non-null object
user              11 non-null object
user_bio          11 non-null object
user_id           11 non-null int64
emotion           11 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 1.3+ KB


# Data Normalization

In [26]:
df_for_norm = data_df[data_df['language'] == 'en']

In [27]:
clean_text = df_for_norm['clean_text'].tolist()
mentions = df_for_norm['at_mentions'].tolist()
hashtags = df_for_norm['hashtags'].tolist()
tweet_id = df_for_norm['tweet_id'].tolist()

In [28]:
def remove_mh(in_string, mentions, hashtags):
    """Remove at_mentions and hashtags from tweets.
    """
    mentions = ['@'+s for s in mentions]
    hashtags = ['#'+s for s in hashtags]
    stopwords = mentions+hashtags
    querywords = in_string.split()
    resultwords  = [word for word in querywords if word not in stopwords]
    result = ' '.join(resultwords)
    return result

In [29]:
def normalize(in_string, mentions, hashtags):
    """Normalize tweets' text.
    """
    in_string = remove_mh(in_string, mentions, hashtags)
    result = in_string.lower() # To Lower
    result = result.translate(str.maketrans(' ',' ',string.punctuation)) # Remove Puntuation
    result = result.strip() # Remove White Space
    lemmatizer=WordNetLemmatizer() # Lemmatization
    stop_words = set(stopwords.words('english')) # Remove Stop Words, at_mentions and hashtags
    tokens = word_tokenize(result)
    result = [lemmatizer.lemmatize(i) for i in tokens if not i in stop_words]
    return result

In [30]:
list_norm = []

In [31]:
for i in range(len(clean_text)):
    dic_norm = {
        "tweet_id": tweet_id[i],
        "tokens": normalize(clean_text[i], mentions[i], hashtags[i])
    }
    list_norm.append(dic_norm)

In [32]:
df_norm = pd.DataFrame(list_norm)

In [33]:
df_norm.head()

,tokens,tweet_id
0,"[im, getting, sick, wordscapes, daily, puzzle,...",1177074894867259392
1,"[’, level, 350, wordscapes, case, ’, wondering...",1177060826794139648
2,"[im, gon, na, finish, wordscapes, daily, puzzl...",1177053151406542848
3,"[ciara, played, 2000, game, wordscapes, ’, bel...",1177049574143352832
4,"[come, play, wordscapes, fun, combination, wor...",1177019961233084416


In [34]:
df_norm_clean = df_norm.tokens.apply(pd.Series) \
 .merge(df_norm, right_index = True, left_index = True) \
 .drop(["tokens"], axis = 1) \
 .melt(id_vars = ['tweet_id'], value_name = "tokens") \
 .drop("variable", axis = 1) \
 .dropna()

In [35]:
df_norm_clean.head()

,tweet_id,tokens
0,1177074894867259392,im
1,1177060826794139648,’
2,1177053151406542848,im
3,1177049574143352832,ciara
4,1177019961233084416,come


In [36]:
df_freq = df_norm_clean[["tokens","tweet_id"]].groupby("tokens").count()

In [37]:
df_freq.head()

,tweet_id
tokens,
1,13
10,2
11,2
114,1
116,1


# Data Exportation

In [38]:
tweet_data = data_df.drop(['if_retweet'], axis=1)

In [39]:
tweet_data = tweet_data[tweet_data['type'] != 'retweet']

In [40]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 261 entries, 2019-09-26 04:18:18 to 2019-09-16 17:36:48
Data columns (total 14 columns):
at_mentions       261 non-null object
clean_text        261 non-null object
favorite_count    261 non-null int64
hashtags          261 non-null object
language          261 non-null object
polarity          261 non-null float64
retweet_count     261 non-null int64
text              261 non-null object
tweet_id          261 non-null int64
type              261 non-null object
user              261 non-null object
user_bio          261 non-null object
user_id           261 non-null int64
emotion           261 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 30.6+ KB


In [41]:
frames = [tweet_data, retweet_df]

In [42]:
df_comb = pd.concat(frames)

In [43]:
df_comb.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 272 entries, 2019-09-26 04:18:18 to 2019-09-16 23:10:01
Data columns (total 14 columns):
at_mentions       272 non-null object
clean_text        272 non-null object
favorite_count    272 non-null int64
hashtags          272 non-null object
language          272 non-null object
polarity          272 non-null float64
retweet_count     272 non-null int64
text              272 non-null object
tweet_id          272 non-null int64
type              272 non-null object
user              272 non-null object
user_bio          272 non-null object
user_id           272 non-null int64
emotion           272 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 31.9+ KB


In [44]:
with pd.ExcelWriter('twitter.xlsx') as writer: 
    retweet_df.to_excel(writer, sheet_name='Retweet')
    data_df.to_excel(writer, sheet_name='Raw Data')
    df_comb.to_excel(writer, sheet_name='Combination')
    df_freq.to_excel(writer, sheet_name='Frequency')
    df_norm_clean.to_excel(writer, sheet_name='Tokens')